## OpenVINO optimizations for Text classification task


## Import the packages needed for successful execution

In [17]:
from transformers import AutoConfig, AutoTokenizer, default_data_collator
from datasets import load_dataset, load_metric
from optimum.intel.openvino import OVAutoModelForSequenceClassification

from torch.utils.data import DataLoader

from tqdm import tqdm

### Instructions on conversion to OpenVINO
We will use the OpenVINO™ Integration with Optimum module to convert the sentiment classification model to an OpenVINO model object. <br>
We will then use Huggingface datasets and metric to evaluate the converted model.

In [37]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
ov_model = OVAutoModelForSequenceClassification.from_pretrained(model_name, config=config, from_pt=True)
ov_model.save_pretrained('saved_model')

Loading model from PT file, inputs: None


[W shape_type_inference.cpp:419] Warning: Constant folding in symbolic shape inference fails: index_select(): Index is supposed to be a vector
Exception raised from index_select_out_cpu_ at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:758 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x42 (0x7f5f89458302 in /home/dkarkada/miniconda3/envs/optimumtests/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: at::native::index_select_out_cpu_(at::Tensor const&, long, at::Tensor const&, at::Tensor&) + 0x2a9 (0x7f5f77db9b89 in /home/dkarkada/miniconda3/envs/optimumtests/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #2: at::native::index_select_cpu_(at::Tensor const&, long, at::Tensor const&) + 0x60 (0x7f5f77dbc5e0 in /home/dkarkada/miniconda3/envs/optimumtests/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #3: <unknown function> + 0x19acf62 (0x7f5f78531f62 in /home/dkarkada/miniconda3/envs/optimumtests/lib/pyth

### Preprocess function for the dataset


In [12]:
def preprocess_function(examples):
    result =  tokenizer(examples['sentence'], padding="max_length", max_length=128, truncation=True)
    result["labels"] = examples["label"]
    return result

### Evaluating the model performance

In [39]:
dataset = load_dataset("sst2")
metric = load_metric('f1')

dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
val_dataloader = DataLoader(
       dataset['validation'], shuffle=True, collate_fn=default_data_collator
    )

for idx, batch in enumerate(tqdm(val_dataloader, desc="Looping over validation data")):
    outputs = ov_model(input_ids=batch['input_ids'].numpy(), attention_mask=batch['attention_mask'].numpy())
    preds = outputs[0].argmax()
    references = batch['labels'].numpy()
    metric.add_batch(predictions=[preds], references=[references])

ov_score = metric.compute()
print(f'Score for SST2 dataset with OV Optimum: {ov_score}')

06/16/2022 00:13:20 - WARNING - datasets.load - Couldn't find a directory or a dataset named 'sst2' in this version. It was picked from the master branch on github instead.
06/16/2022 00:13:20 - WARNING - datasets.builder - Using custom data configuration default
06/16/2022 00:13:20 - WARNING - datasets.builder - Reusing dataset sst2 (/home/dkarkada/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
100%|██████████| 3/3 [00:00<00:00, 434.57it/s]
06/16/2022 00:13:20 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/dkarkada/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-e7c8b4634b8c30e9.arrow
06/16/2022 00:13:21 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/dkarkada/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-75db2d1b4bea1

Score for SST2 dataset with OV Optimum: {'accuracy': 0.9105504587155964}


### Benchmark the converted model using the benchmark app
The OpenVINO toolkit provides a benchmarking application to gauge the platform specific runtime performance that can be obtained under optimal configuration parameters for a given model. For more details refer to: https://docs.openvino.ai/latest/openvino_inference_engine_tools_benchmark_tool_README.html

In [38]:
base_model_name = 'saved_model/ov_model.xml'
print('Benchmark OpenVINO model using the benchmark app')
! benchmark_app -m "$base_model_name" -d CPU -api async -t 10 -hint latency

Benchmark OpenVINO model using the benchmark app
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[Step 1/11] Parsing and validating input arguments
[ WARNING ]  -nstreams default value is determined automatically for a device. Although the automatic selection usually provides a reasonable performance, but it still may be non-optimal for some cases, for more information look at README. 
[Step 2/11] Loading OpenVINO
[ INFO ] OpenVINO:
         API version............. 2022.1.0-7019-cdb9bec7210-releases/2022/1
[ INFO ] Device info
         CPU
         openvino_intel_cpu_plugin version 2022.1
         Build................... 2022.1.0-7019-cdb9bec7210-releases/2022/1

[Step 3/11] Setting device configuration
[Step 4/1